In [1]:
!pip install fastembed


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from fastembed import TextEmbedding

q = "I just discovered the course. Can I join now?"
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
embeddings = list(model.embed(q))
print(len(embeddings[0]))

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


512


In [3]:
print(min(embeddings[0]))

-0.11726373885183883


In [4]:
import numpy as np
np.linalg.norm(embeddings)

np.float64(1.0)

In [5]:
embeddings[0].dot(embeddings[0])

np.float64(1.0000000000000002)

In [6]:
doc = 'Can I still join the course after the start date?'
doc_em = list(model.embed(doc))
embeddings[0].dot(doc_em[0])

np.float64(0.9008528895674548)

In [7]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
all_sim = []
for i in documents:
    # print(i['text'])
    text_em = list(model.embed(i['text']))
    sim = text_em[0].dot(embeddings[0])
    all_sim.append(sim)

print(all_sim)
print(max(all_sim))


[np.float64(0.7629684696540238), np.float64(0.8182378150042889), np.float64(0.8085397398734189), np.float64(0.7133079015686243), np.float64(0.7304499234333598)]
0.8182378150042889


In [9]:
all_sim = []
for i in documents:
    # print(i['text'])
    full_text = i['question'] + ' ' + i['text']
    text_em = list(model.embed(full_text))
    sim = text_em[0].dot(embeddings[0])
    all_sim.append(sim)

print(all_sim)
print(max(all_sim))


[np.float64(0.8514543236908068), np.float64(0.8436594159113068), np.float64(0.8408287048502558), np.float64(0.7755157969663908), np.float64(0.8086007795043938)]
0.8514543236908068


In [10]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [11]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")

In [12]:
from fastembed import TextEmbedding

import json

# EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["model"] == "BAAI/bge-small-en":
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


In [14]:
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

collection_name = "hw2"

# # Create the collection with specified vector parameters
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(
#         size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
#         distance=models.Distance.COSINE  # Distance metric for similarity search
#     )
# )

In [15]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
            id=id,
            vector=models.Document(text=(doc['question'] + ' ' +doc['text']), model=model_handle), 
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
    points.append(point)

    id += 1